# ECON 490: Combining Datasets (7)
---
## Prerequisites: 
---
1. Import datasets in csv and dta format. 
2. Create new variables for a variety of purposes. 
3. Use group_by and other functions to conduct group level analysis.

## Learning Objectives:
---
- Append new observations to an already existing dataset using `rbind`.
- Merge variables and their values from one dataset into another using `cbind`, `left_join`, `inner_join`, and `full_join`.

In [1]:
library(haven)
library(tidyverse)

Warning message:
“package ‘haven’ was built under R version 4.1.3”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

Warning message:
“package ‘ggplot2’ was built under R version 4.1.3”
Warning message:
“package ‘tibble’ was built under R version 4.1.3”
Warning message:
“package ‘dplyr’ was built under R version 4.1.3”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



We'll continue working with the fake data dataset introduced in the previous lecture. Recall that this dataset is simulating information of workers in the years 1982-2012 in a fake country where a training program was introduced in 2003 to boost their earnings. 

In [ ]:
fake_data <- read_csv("../econ490-stata/fake_data.csv")  # change me!

Since we are working with multiple datasets in this module, we will also import the region year dataset below. This dataset is much smaller and gives the average log earnings and total number of people employed among each region and in each year.

In [ ]:
region_year_data <- read_dta("../econ490-stata/region_year_data.dta") # change me!

Often we will need to draw on data from multiple datasets such as these. Most of the time, these datasets will be available for download in different files (each for a given year, month, country, etc.) and may store different variables or observations. Thus, if we want to compile them we need to combine them into the same data frame.

There are two key ways of combining data, each reflecting different goals:

1. When we want to add more observations from another dataset into our existing dataset, we call this **appending** data.
    * If you think of a dataset as a spreadsheet, this is like taking one dataset and "pasting" it into the bottom of another to add more observations. We do this when two datasets have identical columns/variables (so that we can stack them vertically).
2. When we want to add new variables and their data from another dataset into our existing dataset, we call this **merging** data.
    * This is like looking up values in a table and then adding a column; in Excel, this is called a `VLOOKUP`. Importantly, we can only merge data that share a common column or key to  identify observations with particular values. For example, if we want to merge in data from a different year but for the same people (observations) as those we are currently working with, datasets will usually have an identifying number for the person that functions as our key when merging.

## 7.1: Appending Datasets
---

Let's say that our `fake_data` dataset is inexplicably missing 3 observations for worker 1; specifically, the earnings for this worker for the years 2003, 2005, and 2007 are missing. However, let's say these observations exist in another dataset, `missing_data`, which we can append to our `fake_data` dataset since it contains all of the same variables. We can inspect this small dataframe below.

In [2]:
missing_data <- data.frame(workerid = c(1, 1, 1), year = c(2003, 2005, 2007), sex = c("M", "M", "M"), 
                           birth_year = c(1944, 1944, 1944), age = c(59, 61, 63), start_year = c(1997, 1997, 1997),
                           region = c(1, 1, 1), treated = c(0, 0, 0), earnings = c(30000, 35000, 36000))

missing_data

workerid,year,sex,birth_year,age,start_year,region,treated,earnings
<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,2003,M,1944,59,1997,1,0,30000
1,2005,M,1944,61,1997,1,0,35000
1,2007,M,1944,63,1997,1,0,36000


To append these four rows to the bottom of our dataset, we can simply use the `rbind` function. This function allows us to bind together datasets vertically, with the dataset specified second being placed directly underneath the dataset specified first. In this way, we can combine datasets vertically if they share the same column names. 

In [ ]:
fake_data <- rbind(fake_data, missing_data)
tail(fake_data)

This is a fast way of concatenating datasets vertically. We can see that it also does not require us to have a designated "master" and "using" dataset. We can have both datasets stored in our notebook and view them simultaneously, making the process of appending datasets simpler, especially if we want to check for identical column names or missing values.

## 7.2: Merging Datasets 
---
More often, however, we want to combine datasets horizontally. This means matching existing observations between datasets along specific variables, typically in order to add more information about existing participants to our current dataset. This process is known as merging or joining data, and it is more complicated and has more cases than simply appending data.

### 7.2.1: Simple merge with `cbind`
The easiest type of merge we can do is a simple merge with the `cbind` function. This is equivalent to performing a 1:1 merge in Stata between two datasets with equal numbers of observations. In this way, it is identical to stacking datasets vertically like with `rbind`, we just stack the datasets horizontally. It functions more like a merge since we must use a common "key" or index to 

### 1. Check the dataset's unique identifiers 
The key to merging datasets is to understand what are the variables that *uniquely* identify each observation.

In [3]:
%browse 10

,workerid,year,sex,birth_year,age,start_year,region,treated,earnings
1,1,1999,M,1944,55,1997,1,0,39975.008
2,1,2001,M,1944,57,1997,1,0,278378.06
3,2,2001,M,1947,54,2001,4,0,18682.6
4,2,2002,M,1947,55,2001,4,0,293336.41
5,2,2003,M,1947,56,2001,4,0,111797.26
6,3,2005,M,1951,54,2005,5,0,88351.672
7,3,2010,M,1951,59,2005,5,0,46229.574
8,4,1997,M,1952,45,1997,5,1,24911.029
9,4,2001,M,1952,49,1997,5,1,9908.3623
10,5,2009,M,1954,55,1998,2,1,137207.34


For example, it seems like each observation in the fake_data dataset is identified by the variables workerid and year (worker-year pair). We need to check whether this is true or not using the `duplicates report` command.

In [4]:
duplicates report workerid year


Duplicates in terms of workerid year

--------------------------------------
   copies | observations       surplus
----------+---------------------------
        1 |      2861772             0
--------------------------------------


What this table shows is that there are 2861772 workerid-year combination (which is exactly equal to all of our observations). This means that every observation we have corresponds to a worker in a particular year. 

Let's take a look at a different dataset now.

In [5]:
use region_year_data, clear

In [6]:
%browse 10

,year,region,avg_log_earnings,total_employment
1,1998,1,10.506687,30004
2,1999,1,10.513171,31367
3,2000,1,10.511585,33429
4,2001,1,10.550608,34547
5,2002,1,10.529206,35503
6,2003,1,10.615291,35809
7,2004,1,10.558952,36161
8,2005,1,10.538996,36966
9,2006,1,10.511196,38161
10,2007,1,10.525853,38051


In this case, it seems that every observation corresponds to a region and year combination. Again, we can use `duplicates report` to see if the variables `region` and `year` uniquely identify all observations.

In [7]:
duplicates report region year


Duplicates in terms of region year

--------------------------------------
   copies | observations       surplus
----------+---------------------------
        1 |           70             0
--------------------------------------


Indeed! The table shows that there is not a single case of repeated copies of some observation. Hence, we will refer to these variables as the `unique identifiers`.

### 1. Master and Using Datasets

When merging data we need to decide which dataset will be the dataset (Stata refers to this dataset as `master`) and which will be secondary dataset. The secondary dataset  is were we will get the specific variables we want to duplicate in the `master` dataset and it is refered to as the `using` data.

### 2. Matching observations 
There are three main ways to match observations. The first case is when both observations share the same unique identifiers, so one observation in the master dataset is matched to one observation in the using dataset (reffered as `1:1` merge). The other two cases arise when you match multiple observations in the master dataset to one observation in the using dataset (referred as `m:1` merge). If it is the case that one observation in the master dataset is matched to multiple observations in the using dataset this is known as a `1:m` merge.

### Merging
Once we know the unique identifiers, the master and using datasets and what type of match we are doing we are able to merge the datasets. 

We begin by choosing the master dataset and having it opened in the current Stata session. For the sake of showing an example, let's suppose we want to set fake_data as the `master` dataset, and use region-year  as the `using` dataset. Like we said the fake_data's unique identifiers are workerid and year while the region-year's unique identifiers are region and year. The variables we use to link both datasets have to be the unique identifiers that are present in both datasets. Because `workerid` does not exist in the region-level dataset, we will use variable  `region` and `year` to merge the datasets. 

This would mean that for every region in the using dataset there will be many observations in the individual level (master) dataset to be matched. Therefore, this will be a `m:1` merge.  


In [ ]:
use fake_data, clear

In [10]:
merge m:1 region year using region_year_data


    Result                           # of obs.
    -----------------------------------------
    not matched                       406,963
        from master                   406,963  (_merge==1)
        from using                          0  (_merge==2)

    matched                         2,454,809  (_merge==3)
    -----------------------------------------


Let's analyze the table above. It says that there were 406,963 observations in the master data couldn't be matched to any observation in the using dataset. This is due to the fact that our dataset at the region-year level does not have information for some years. 

Furthermore, the previous table shows that every observation from the using dataset got matched to some observation in the master dataset. The total number of matched observations is roughly 2.5 million. All of this information gets recorded into a new varible named `_merge`. Because of this, it is good practice to write `cap drop _merge` before running a merge command or use the `nogen` option of this command. 

Would we get the same results if we switched the master and using datasets?

In [13]:
use region_year_data, clear
merge 1:m region year using fake_data




    Result                           # of obs.
    -----------------------------------------
    not matched                       406,963
        from master                         0  (_merge==1)
        from using                    406,963  (_merge==2)

    matched                         2,454,809  (_merge==3)
    -----------------------------------------


Indeed, we get the same information. We typically want to restrict to observations that were correctly matched across datasets.

In [14]:
keep if _merge==3

(406,963 observations deleted)


It should finally be noted that there is a function called `merge` which can accomplish all of the above merges in a single function. However, it is much longer when operating with large datasets, so we didn't cover it. If you wish to look at the documentation for merge, you can run the code below.

In [ ]:
?merge

## 7.3: Wrap up
---

In this module we learned how to combine different datasets. The most important lesson we should take away from this module is that, When we want to merge different variables (columns) from one dataset to another we use the command `merge`. If we want to add observations (rows) from one dataset to another we use the command `append`.

Also, for merging, do not forget to identify the which dataset is the `master` and which is the `using`, which variables are you merging and what are the unique identifiers.